# **AUTORIZANDO ACESSO AO DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **EXTRAÇÃO DAS TABELAS VIA DRIVE**

In [ ]:
import pandas as pd

# --- VERIFIQUE OS CAMINHOS AQUI ---
# Garanta que o nome da sua pasta no Google Drive está correto em todos os caminhos.
caminho_base = '/content/drive/MyDrive/Big Data/projeto_bigdata/'

arquivos = {
    'customers': f'{caminho_base}customers.csv',
    'products': f'{caminho_base}products.csv',
    'orders': f'{caminho_base}orders.csv',
    'order_items': f'{caminho_base}order_items.csv',
    'product_reviews': f'{caminho_base}product_reviews.csv'
}

# Dicionário para armazenar nossos DataFrames
dataframes = {}

print("Iniciando o carregamento de todas as 5 tabelas...")

try:
    for nome, caminho in arquivos.items():
        print(f"\nCarregando a tabela: '{nome}'...")
        # Carrega o CSV para um DataFrame e o armazena no dicionário
        dataframes[nome] = pd.read_csv(caminho)
        print(f"Tabela '{nome}' carregada com sucesso!")

    # Atribuindo os dataframes a variáveis individuais para facilitar o uso
    df_customers = dataframes['customers']
    df_products = dataframes['products']
    df_orders = dataframes['orders']
    df_order_items = dataframes['order_items']
    df_product_reviews = dataframes['product_reviews']

    print("\n" + "="*50)
    print("CARREGAMENTO CONCLUÍDO COM SUCESSO!")
    print("="*50)
    print("Abaixo está um resumo de cada tabela carregada.\n")

    # Mostra um resumo de cada DataFrame para verificação
    for nome, df in dataframes.items():
        print(f"\n--- Informações da Tabela: {nome} ---")
        df.info()
        print("-" * (30 + len(nome)))

except FileNotFoundError as e:
    print(f"\nERRO: Um dos arquivos não foi encontrado!")
    print(f"Detalhe do erro: {e}")
    print("Por favor, verifique se todos os 5 arquivos CSV estão na pasta correta no Google Drive e se os nomes estão corretos.")


Iniciando o carregamento de todas as 5 tabelas...

Carregando a tabela: 'customers'...
Tabela 'customers' carregada com sucesso!

Carregando a tabela: 'products'...
Tabela 'products' carregada com sucesso!

Carregando a tabela: 'orders'...
Tabela 'orders' carregada com sucesso!

Carregando a tabela: 'order_items'...
Tabela 'order_items' carregada com sucesso!

Carregando a tabela: 'product_reviews'...
Tabela 'product_reviews' carregada com sucesso!

CARREGAMENTO CONCLUÍDO COM SUCESSO!
Abaixo está um resumo de cada tabela carregada.


--- Informações da Tabela: customers ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   customer_id  int64 
 1   name         object
 2   email        object
 3   gender       object
 4   signup_date  object
 5   country      object
dtypes: int64(1), object(5)
memory usage: 91.6+ MB
---------------------------------------

--- Informações

# **TRATAMENTO E VERIFICAÇÃO DAS TABELAS**

In [ ]:
import pandas as pd

print("="*50)
print("Iniciando a Limpeza Unificada de Todas as 5 Tabelas")
print("="*50)

# Dicionário com todos os 5 DataFrames que precisam de limpeza
dataframes_para_limpar = {
    "customers": df_customers,
    "products": df_products,
    "orders": df_orders,
    "order_items": df_order_items,
    "product_reviews": df_product_reviews
}

# Mapeamento de todas as colunas de data que precisam de conversão
colunas_de_data = {
    "customers": ["signup_date"],
    "orders": ["order_date"],
    "product_reviews": ["review_date"]
}

# Loop principal de limpeza
for nome, df in dataframes_para_limpar.items():
    print(f"\n--- Processando a tabela: '{nome}' ---")
    linhas_antes = len(df)

    # 1. Converter colunas de data (se aplicável)
    if nome in colunas_de_data:
        for coluna in colunas_de_data[nome]:
            print(f"  - Convertendo a coluna de data: '{coluna}'...")
            df[coluna] = pd.to_datetime(df[coluna])

    # 2. Remover linhas com valores nulos
    df.dropna(inplace=True)

    # 3. Remover linhas duplicadas
    df.drop_duplicates(inplace=True)

    linhas_depois = len(df)
    print(f"  - Limpeza concluída. {linhas_antes - linhas_depois} linhas foram removidas.")


print("\n\n" + "="*50)
print("LIMPEZA DE TODAS AS TABELAS CONCLUÍDA!")
print("Abaixo, o resumo final de todas as 5 tabelas tratadas.")
print("="*50)

# Verificação final de todas as 5 tabelas
for nome, df in dataframes_para_limpar.items():
    print(f"\n--- Informações Finais da Tabela: {nome} ---")
    df.info()
    print("-" * (35 + len(nome)))

Iniciando a Limpeza Unificada de Todas as 5 Tabelas

--- Processando a tabela: 'customers' ---
  - Convertendo a coluna de data: 'signup_date'...
  - Limpeza concluída. 0 linhas foram removidas.

--- Processando a tabela: 'products' ---
  - Limpeza concluída. 0 linhas foram removidas.

--- Processando a tabela: 'orders' ---
  - Convertendo a coluna de data: 'order_date'...
  - Limpeza concluída. 0 linhas foram removidas.

--- Processando a tabela: 'order_items' ---
  - Limpeza concluída. 0 linhas foram removidas.

--- Processando a tabela: 'product_reviews' ---
  - Convertendo a coluna de data: 'review_date'...
  - Limpeza concluída. 0 linhas foram removidas.


LIMPEZA DE TODAS AS TABELAS CONCLUÍDA!
Abaixo, o resumo final de todas as 5 tabelas tratadas.

--- Informações Finais da Tabela: customers ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   cust

# **ENVIO DE DADOS PARA O GOOGLE BIG QUERY**

In [ ]:
!pip install pandas-gbq -q

# Autenticar conta
from google.colab import auth
auth.authenticate_user()
print("Autenticação concluída.")

project_id = "big-data-fatesg"

# Dicionário com os 5 DataFrames limpos
todos_os_dataframes = {
    "customers": df_customers,
    "products": df_products,
    "orders": df_orders,
    "order_items": df_order_items,
    "product_reviews": df_product_reviews
}

# Nome do dataset que será criado no BigQuery para organizar as tabelas
dataset_name = "loja_virtual_tratado"

print(f"\nIniciando o envio de dados para o BigQuery no projeto '{project_id}'...")

# Loop para enviar cada DataFrame para o BigQuery
for nome, df in todos_os_dataframes.items():
    destination_table = f"{dataset_name}.{nome}"
    print(f"  - Enviando a tabela '{nome}' para '{destination_table}'...")

    # Comando para enviar o DataFrame
    df.to_gbq(destination_table=destination_table,
              project_id=project_id,
              if_exists='replace') # 'replace' apaga a tabela se ela já existir

    print(f"  -> Tabela '{nome}' enviada com sucesso!")

print("\n" + "="*50)
print("Todos os dados foram carregados no BigQuery.")
print("="*50)

Autenticação concluída.

Iniciando o envio de dados para o BigQuery no projeto 'big-data-fatesg'...
  - Enviando a tabela 'customers' para 'loja_virtual_tratado.customers'...


/tmp/ipython-input-967506951.py:32: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destination_table,
100%|██████████| 1/1 [00:00<00:00, 7584.64it/s]
/tmp/ipython-input-967506951.py:32: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destination_table,


  -> Tabela 'customers' enviada com sucesso!
  - Enviando a tabela 'products' para 'loja_virtual_tratado.products'...


100%|██████████| 1/1 [00:00<00:00, 7943.76it/s]
/tmp/ipython-input-967506951.py:32: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destination_table,


  -> Tabela 'products' enviada com sucesso!
  - Enviando a tabela 'orders' para 'loja_virtual_tratado.orders'...


...

  -> Tabela 'orders' enviada com sucesso!
  - Enviando a tabela 'order_items' para 'loja_virtual_tratado.order_items'...


100%|██████████| 1/1 [00:00<00:00, 11586.48it/s]
/tmp/ipython-input-967506951.py:32: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=destination_table,


  -> Tabela 'order_items' enviada com sucesso!
  - Enviando a tabela 'product_reviews' para 'loja_virtual_tratado.product_reviews'...


100%|██████████| 1/1 [00:00<00:00, 11008.67it/s]

  -> Tabela 'product_reviews' enviada com sucesso!

FASE 3 CONCLUÍDA! Todos os dados foram carregados no BigQuery.
